NYC-TAXI-EDA-FEATURE-ENGINEERING<br>
https://www.kaggle.com/frednavruzov/nyc-taxi-eda-feature-engineering

In [1]:
import pandas as pd
import numpy as np
import sympy
import datetime as dt
import time
from math import *
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from ipyleaflet import *
import folium
import json
import geopy.distance
from haversine import haversine
from tqdm import tqdm_notebook

sns.set()
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

import platform
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)

In [2]:
train = pd.read_csv("../dataset/train.csv") # 각자 데이터셋의 폴더

FileNotFoundError: File b'../dataset/train.csv' does not exist

### data에 대한 기본적인 탐색

- traing data의 row 수와 column 수

In [ ]:
print("traing data의 row 수 : {}, column 수 : {}".format(train.shape[0], train.shape[1]))

In [ ]:
train.info()

In [3]:
train.describe().round(2)

NameError: name 'train' is not defined

In [4]:
# taxi['count'] = 1

# 전처리

- 아웃라이어 제거
1. 시간
    - pickup_datetime
2. 위치
3. 승객수

### 아웃라이어 제거
- 시간
- 승객수
- 위치

In [5]:
# 운행 시간 0, 2시간 벗어나는 데이터 제거

# taxi = taxi[taxi["trip_duration"] >= 0]
# taxi = taxi[taxi["trip_duration"] <= 60*60*2]

In [6]:
# 승객 수 0명 제거

# taxi = taxi[taxi["passenger_count"] != 0]

## pickup_datetime 살피기

In [7]:
# pd.to_datetime을 해줘야 pickup_datetime coulmn의 data type이 datetime으로 됨
pickup_datetime_dt = pd.to_datetime(train["pickup_datetime"])
dropoff_datetime_dt = pd.to_datetime(train["dropoff_datetime"])

NameError: name 'train' is not defined

In [8]:
train["pickup_datetime"] = pickup_datetime_dt
train["dropoff_datetime"] = dropoff_datetime_dt

train["pickup_date"] = train["pickup_datetime"].dt.date
train["dropoff_date"] = train["dropoff_datetime"].dt.date

train["pickup_month"] = train["pickup_datetime"].dt.month
train["dropoff_month"] = train["dropoff_datetime"].dt.month

train["pickup_weekday"] = train["pickup_datetime"].dt.weekday
train["dropoff_weekday"] = train["dropoff_datetime"].dt.weekday

train["pickup_hour"] = train["pickup_datetime"].dt.hour
train["dropoff_hour"] = train["dropoff_datetime"].dt.hour

NameError: name 'pickup_datetime_dt' is not defined

In [9]:
train.info()

NameError: name 'train' is not defined

In [10]:
train.describe().round(2)

NameError: name 'train' is not defined

### EDA of pickup_datetime in train data
- year : 2016년 
- month : 1~7월
- hour : 0~23시
- weekday : 월요일~일요일

In [11]:
# # year
# print("데이터가 기록된 연도")
# print("가장 오래된 데이터의 기록 연도: {}년".format(taxi_df1["pickup_datetime"].dt.year.min()))
# print("가장 최신 데이터의 기록 연도: {}년".format(taxi_df1["pickup_datetime"].dt.year.max()))
# print('')

# # month
# print("데이터가 기록된 달")
# print("가장 오래된 데이터의 기록된 달: {}월".format(taxi_df1["pickup_month"].min()))
# print("가장 최신 데이터가 기록된 달: {}월".format(taxi_df1["pickup_month"].max()))

In [12]:
train[train["trip_duration"] >= 60*60*24]

NameError: name 'train' is not defined

In [ ]:
train = train[train["trip_duration"] <= 60*60*24]

In [ ]:
plt.figure(figsize=(7, 5))
sns.distplot(train["trip_duration"], color="r")
plt.xlabel("Trip Duration")
plt.show()

In [ ]:
sns.set()
plt.figure(figsize=(7, 5))
sns.distplot(np.log(train['trip_duration']+1), color="r")
plt.xlabel("Log of Trip Duration")
plt.show()

In [ ]:
plt.figure(figsize=(7, 4))
sns.countplot(x="vendor_id", data=train, palette="husl")
plt.title("Vendor Distribution", fontsize=13)
plt.xlabel("Vendor")
plt.ylabel("Number of Trips")
plt.show()

In [ ]:
plt.figure(figsize=(7, 4))
sns.countplot(x="store_and_fwd_flag", data=train, palette="husl")
plt.title("Store & FWD Flag Distribution", fontsize=13)
plt.xlabel("Store & FWD Flag")
plt.ylabel("Number of Trips")
plt.show()

In [ ]:
plt.figure(figsize=(13, 4))

plt.subplot(121)
sns.countplot(x="pickup_month", data=train, palette="husl")
plt.title("Pickups Month Distribution", fontsize=13)
plt.xlabel("Pickup Months (January-June)")
plt.ylabel("Number of Trips")

plt.subplot(122)
sns.countplot(x="pickup_month", data=train, palette="husl", hue="vendor_id")
plt.title("Pickups Month Distribution", fontsize=13)
plt.xlabel("Pickup Months (January-June)")
plt.ylabel("Number of Trips")
plt.legend(loc=(1.04,0))

plt.show()

In [ ]:
plt.figure(figsize=(13, 4))

plt.subplot(121)
sns.countplot(x="pickup_weekday", data=train, palette="husl")
plt.title("Pickups Weekday Distribution", fontsize=13)
plt.xlabel("Pickup Weekday (Mon-Sun)")
plt.ylabel("Number of Trips")

plt.subplot(122)
sns.countplot(x="pickup_weekday", data=train, palette="husl", hue="vendor_id")
plt.title("Pickups Weekday Distribution", fontsize=13)
plt.xlabel("Pickup Weekday (Mon-Sun)")
plt.ylabel("Number of Trips")
plt.legend(loc=(1.04,0))

plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(x="pickup_hour", data=train, palette="husl")
plt.title("Pickups Hour Distribution", fontsize=13)
plt.xlabel("Pickup Hours (0-23)")
plt.ylabel("Number of Trips")
plt.show()

In [ ]:
# Trip Duration의 경우, 상위 4개의 값이 워낙 큼. mean이 아닌 mode로 하면 차이는 더 많이 남)
# boxplot으로 보여주는 것도 좋을 것 같음 (단, log 처리하지 않으면 아웃라이어에 영향을 많이 받을 것)

data = train.loc[:, ["pickup_hour", "trip_duration"]].groupby("pickup_hour").mean()

plt.figure(figsize=(17, 4))

plt.subplot(121)
sns.barplot(x=data.index, y=data.trip_duration, data=data, palette="husl")
plt.title("Pickups Hours & Trip Duration", fontsize=13)
plt.xlabel("Pickup Hours (0-23)")
plt.ylabel("Trip Duration")


plt.subplot(122)
sns.barplot(x=data.index, y=np.log(data.trip_duration+1), data=data, palette="husl")
plt.title("Pickups Hours & Log Trip Duration", fontsize=13)
plt.xlabel("Pickup Hours (0-23)")
plt.ylabel("Log of Trip Duration")

plt.show()

In [ ]:
data = train.loc[:, ["pickup_weekday", "trip_duration"]].groupby("pickup_weekday").mean()

plt.figure(figsize=(17, 4))

plt.subplot(121)
sns.barplot(x=data.index, y=data.trip_duration, data=data, palette="husl")
plt.title("Pickups Weekday & Trip Duration", fontsize=13)
plt.xlabel("Pickup Weekday (Mon-Sun)")
plt.ylabel("Trip Duration")

plt.subplot(122)
sns.barplot(x=data.index, y=np.log(data.trip_duration+1), data=data, palette="husl")
plt.title("Pickups Weekday & Log Trip Duration", fontsize=13)
plt.xlabel("Pickup Weekday (Mon-Sun)")
plt.ylabel("Log of Trip Duration")

plt.show()

In [ ]:
plt.figure(figsize=(7, 5))
sns.boxplot(x=train["pickup_weekday"], 
            y=train["trip_duration"].apply(np.log1p), 
            data=train, palette="husl")
plt.title("Pickups Weekday & Log of Trip Duration", fontsize=13)
plt.xlabel("Pickup Weekday (Mon-Sun)")
plt.ylabel("Log of Trip Duration")

plt.show()

In [ ]:
data = train.loc[:, ["pickup_month", "trip_duration"]].groupby("pickup_month").mean()

plt.figure(figsize=(17, 4))

plt.subplot(121)
sns.barplot(x=data.index, y=data.trip_duration, data=data, palette="husl")
plt.title("Pickups Months & Trip Duration", fontsize=13)
plt.xlabel("Pickup Months (January-June)")
plt.ylabel("Trip Duration")

plt.subplot(122)
sns.barplot(x=data.index, y=np.log(data.trip_duration+1), data=data, palette="husl")
plt.title("Pickups Months & Log Trip Duration", fontsize=13)
plt.xlabel("Pickup Months (January-June)")
plt.ylabel("Log of Trip Duration")

plt.show()

In [ ]:
month = ["January", "February", "March", "April", "May", "June"]
weekday = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
working_day = [0, 1, 2, 3, 4] # Mon-Fri

wd = train.loc[:, ["pickup_weekday", "pickup_month", "trip_duration"]]
wd["working_day"] = wd["pickup_weekday"].isin(working_day)

plt.figure(figsize=(11, 5))
sns.violinplot(x=wd["pickup_month"], 
               y=wd["trip_duration"].apply(np.log1p),
               hue="working_day",
               data=wd, palette="husl")
plt.title("Pickups Month & Log of Trip Duration", fontsize=13)
plt.xlabel("Pickup Month")
plt.ylabel("Log of Trip Duration")
plt.xticks(range(0, 6), month)
plt.show()

In [ ]:
plt.figure(figsize=(11, 7))
sns.countplot(x="pickup_weekday", data=train, hue="pickup_hour")
plt.xlabel("Pickup Weekday (Mon-Sun)")
plt.ylabel("Number of Trips")
plt.xticks(range(0,7), weekday)
plt.legend(loc=(1.04,0))
plt.show()

In [ ]:
plt.figure(figsize=(11,2))
sns.heatmap(data=pd.crosstab(train["pickup_weekday"], 
                             train["pickup_hour"], 
                             values=train["vendor_id"], 
                             aggfunc="count",
                             normalize="index"), cmap="RdPu")

plt.title("Pickup Weekday vs. Hours", fontsize=13)
plt.xlabel("Pickup Hours (0-23)")
plt.ylabel("Pickup Weekday")
plt.yticks(range(0,7), weekday, rotation="horizontal")
plt.show()

In [ ]:
plt.figure(figsize=(11,2))
sns.heatmap(data=pd.crosstab(train["pickup_month"], 
                             train["pickup_hour"], 
                             values=train["vendor_id"], 
                             aggfunc="count",
                             normalize="index"), cmap="RdPu")

plt.title("Pickup Month vs. Hours", fontsize=13)
plt.xlabel("Pickup Hours (0-23)")
plt.ylabel("Pickup Month")
plt.yticks(range(0,6), month, rotation="horizontal")
plt.show()

In [ ]:
plt.figure(figsize=(11,2))
sns.heatmap(data=pd.crosstab(train["pickup_month"], 
                             train["pickup_weekday"], 
                             values=train["vendor_id"], 
                             aggfunc="count",
                             normalize="index"), cmap="RdPu")

plt.title("Pickup Month vs. Weekday", fontsize=13)
plt.xlabel("Pickup Weekday (Mon-Sun)")
plt.ylabel("Pickup Month")
plt.xticks(range(0,7), weekday, rotation=30)
plt.yticks(range(0,6), month, rotation="horizontal")
plt.show()

## 승객수

In [ ]:
train['passenger_count'].value_counts()

In [ ]:
print("가장 적은 탑승 인원: {}명".format(train["passenger_count"].min()))
print("가장 많은 탑승 인원: {}명".format(train["passenger_count"].max()))

In [ ]:
# 승객 수 0명 제거

# train = train[train["passenger_count"] != 0]

In [ ]:
plt.figure(figsize=(7, 5))
sns.countplot(x='passenger_count', data=train, palette="husl")
plt.title("Passenger Count Distribution", fontsize=13)
plt.xlabel("Passenger Count")
plt.ylabel("Number of Trips")
plt.show()

In [ ]:
# 아웃라이어 아직 제거하지 않아서 아웃라이어에 영향을 많이 받음

plt.figure(figsize=(17, 4))

plt.subplot(121)
sns.boxplot(x="passenger_count", 
            y=train["trip_duration"], 
            data=train,
            palette="husl")
plt.title("Passenger Count & Trip Duration", fontsize=13)
plt.xlabel("Passenger Count")
plt.ylabel("Trip Duration")

plt.subplot(122)
sns.boxplot(x="passenger_count", 
            y=train["trip_duration"].apply(np.log1p), 
            data=train,
            palette="husl")
plt.title("Passenger Count & Log Trip Duration", fontsize=13)
plt.xlabel("Passenger Count")
plt.ylabel("Log of Trip Duration")

plt.show()

In [ ]:
f, (ax1, ax2) = plt.subplots(ncols=2, sharey=True, figsize=(14, 4))

sns.boxplot(x="passenger_count", 
            y=train["trip_duration"].apply(np.log1p), 
            hue="vendor_id",
            data=train,
            palette="husl", ax=ax1)

ax1.set_xlabel(""); ax1.set_ylabel("Log of Trip Duration")

sns.boxplot(x="passenger_count", 
            y=train["trip_duration"].apply(np.log1p), 
            hue="store_and_fwd_flag",
            data=train,
            palette="husl", ax=ax2)
ax2.set_xlabel(""); ax2.set_ylabel("")

plt.suptitle("Passenger Count & Log of Trip Duration", y=1.05, fontsize=13)
plt.tight_layout()
f.text(0.5, -0.01, "Passenger Count", ha="center")

plt.show()

In [ ]:
plt.figure(figsize=(11,3))
sns.heatmap(data=pd.crosstab(train["passenger_count"], 
                             train["pickup_hour"], 
                             values=train["vendor_id"], 
                             aggfunc="count",
                             normalize=False), cmap="RdPu")
plt.title("Passenger Count vs. Hours", fontsize=13)
plt.xlabel("Pickup Hours (0-23)")
plt.ylabel("Passenger Count")
plt.show()

In [ ]:
plt.figure(figsize=(11,3))
sns.heatmap(data=pd.crosstab(train["passenger_count"], 
                             train["pickup_month"], 
                             values=train["vendor_id"], 
                             aggfunc="count",
                             normalize=False), cmap="RdPu")
plt.title("Passenger Count vs. Month", fontsize=13)
plt.xlabel("Pickup Month")
plt.ylabel("Passenger Count")
plt.xticks(range(0,6), month, rotation=30)
plt.show()

### EDA 결과
- 승객수 Max 9인, Min 0인 : 0인, 7인이상 삭제 검토 (총 65건)
- 승객수 지표 : 1인 승차객이 전체 약 146만 건 중 103만 건의 비중, 2인 승차객 21만건 / 5인 / 3인 순
- 승객수&승차시각 : 오전 8시 이후 1인 승차객 중심으로 전 시간대 고른 분포
- 승객수&승차요일 : 전반적으로 고른 분포 , 전반적으로 주말보다 수~금 승차건수가 높음
- 승객수&운행시간 : 1인 승객이 평균 1~2분정도 짧게 운행. 전반적으로 1000~1100초 (16분대), 탑승객 없는 택시 유일하게 30분 수준 (공항으로 이동?)

In [ ]:
import folium
import json
geo_path = '../dataset/geojson/state.geo.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [ ]:
# 뉴욕 시 경계를 벗어나는 위/경도 제거

# city_long_border = (-74.03, -73.75)
# city_lat_border = (40.63, 40.85) 

train = train[train['pickup_longitude'] <= -73.75]
train = train[train['pickup_longitude'] >= -74.03]
train = train[train['pickup_latitude'] <= 40.85]
train = train[train['pickup_latitude'] >= 40.63]
train = train[train['dropoff_longitude'] <= -73.75]
train = train[train['dropoff_longitude'] >= -74.03]
train = train[train['dropoff_latitude'] <= 40.85]
train = train[train['dropoff_latitude'] >= 40.63]

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(13,8))

train.plot(kind="scatter", x="pickup_longitude", y="pickup_latitude",
           color="yellow", grid=False, s=.02, alpha=.6, subplots=True, ax=ax1)

ax1.set_title("Pickups")
ax1.set_facecolor("black")

train.plot(kind="scatter", x="dropoff_longitude", y="dropoff_latitude", 
           color="yellow", grid=False, s=.02, alpha=.6, subplots=True, ax=ax2)

ax2.set_title("Dropoffs")
ax2.set_facecolor("black") 

In [ ]:
pickup_lat = tuple(train["pickup_latitude"])
pickup_lng = tuple(train["pickup_longitude"])
dropoff_lat = tuple(train["dropoff_latitude"])
dropoff_lng = tuple(train["dropoff_longitude"])

In [ ]:
pickup_loc = tuple(zip(pickup_lat, pickup_lng))
dropoff_loc = tuple(zip(dropoff_lat, dropoff_lng))

In [ ]:
print(len(pickup_loc))
print(len(dropoff_loc))

In [ ]:
# append가 아니라 전체 컬럼으로 바로 
# vincenty distacne
import geopy.distance
from tqdm import tqdm_notebook

vincenty_distance = []
for i in tqdm_notebook(range(len(pickup_loc))):
    vincenty_distance.append(geopy.distance.vincenty(pickup_loc[i], dropoff_loc[i]).km)

In [ ]:
train.loc[:, "vincenty_distance"] = vincenty_distance
train.loc[:, ["id", "vincenty_distance"]].tail()

In [ ]:
train["log_duration"] = np.log1p(train["trip_duration"])
train["log_vincenty_distance"] = np.log1p(train["vincenty_distance"])

In [ ]:
plt.figure(figsize=(7, 5))
plt.scatter(train.log_vincenty_distance, train.log_duration, alpha=0.05)
plt.ylabel("log(Trip Duration)")
plt.xlabel("log(Vincenty Distance)")
plt.title("log(Vincenty Distance) vs log(Trip Duration)");

In [ ]:
sns.jointplot(x="log_vincenty_distance", y="log_duration", data=train);

In [ ]:
# taxi["pickup_datetime"] = pd.to_datetime(taxi.pickup_datetime)

# taxi.loc[:, "pickup_weekday"] = taxi["pickup_datetime"].dt.weekday
# taxi.loc[:, "pickup_hour_weekofyear"] = taxi["pickup_datetime"].dt.weekofyear
# taxi.loc[:, "pickup_hour"] = taxi["pickup_datetime"].dt.hour
# taxi.loc[:, "pickup_minute"] = taxi["pickup_datetime"].dt.minute
# taxi.loc[:, "pickup_dt"] = (taxi["pickup_datetime"] - taxi["pickup_datetime"].min()).dt.total_seconds()


In [ ]:
train.loc[:, "pickup_week_hour"] = train["pickup_weekday"] * 24 + train["pickup_hour"]
train.loc[:, "avg_speed_h"] = 1000 * train["vincenty_distance"] / train["trip_duration"]

fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, sharey=True, figsize=(13, 6))

ax1.plot(train.groupby("pickup_hour").mean()["avg_speed_h"], 'bo-', lw=2, alpha=0.7)
ax2.plot(train.groupby("pickup_weekday").mean()["avg_speed_h"], 'go-', lw=2, alpha=0.7)
ax3.plot(train.groupby("pickup_week_hour").mean()["avg_speed_h"], 'ro-', lw=2, alpha=0.7)

ax1.set_xlabel("Hour")
ax2.set_xlabel("Weekday")
ax3.set_xlabel("Weekhour")

ax1.set_ylabel("Average Speed (km/h)")
fig.suptitle("Rush hour average traffic speed")
plt.show()

파라미터

- 이동 거리
- 시간대별


- 승/하차 위치 (맨하탄 내부 / 맨하탄 내외부(특히 공항))
- 스피드
- 승객 수

- 가장 큰 영향을 미치는 변수 확인 (아직 안 배움)
- 코드 깔끔히

### 속도와 duration간의 상관관계

In [ ]:
train_hour = train.loc[:, ["pickup_hour", "avg_speed_h", "trip_duration"]]
train_hour["trip_duration"] = train_hour["trip_duration"] / 60
train_hour = train_hour.groupby("pickup_hour").mean()
train_hour.tail()

In [ ]:
train_hour.plot(figsize=(7, 5))
plt.legend(loc='best')
plt.show()

In [ ]:
corr = train_hour.corr()
sns.heatmap(corr, cmap="RdPu");

In [ ]:
np.corrcoef(train_hour.avg_speed_h, train_hour.trip_duration)

In [ ]:
corr_matt = train[["pickup_datetime", "pickup_hour",
                   "pickup_month", "pickup_weekday",
                   "log_vincenty_distance", "log_duration"]]

In [ ]:
corr_matt.head()

In [ ]:
corr_matt = corr_matt.corr()

mask = np.array(corr_matt)
mask[np.tril_indices_from(mask)] = False

In [ ]:
corr_matt.head()

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(14, 7)
sns.heatmap(corr_matt, mask=mask, vmax=.8, annot=True, square=True, cmap="RdPu")
plt.show()

In [ ]:
tmp = train[["trip_duration", "pickup_datetime"]]
tmp.head()